<a href="https://colab.research.google.com/github/prathamsss/Relevant_Words_Analysis_NLP/blob/main/Nanos_Test_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [ ]:
! echo "Installing Magnitude.... (please wait, can take a while)"
! (curl https://raw.githubusercontent.com/plasticityai/magnitude/master/install-colab.sh | /bin/bash 1>/dev/null 2>/dev/null)
! echo "Done installing Magnitude."

Installing Magnitude.... (please wait, can take a while)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   137  100   137    0     0    992      0 --:--:-- --:--:-- --:--:--   985
Done installing Magnitude.


In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [ ]:
# !unzip /content/drive/MyDrive/million-headlines.zip

Archive:  /content/drive/MyDrive/million-headlines.zip
  inflating: abcnews-date-text.csv   


In [ ]:

scrapped_data = urllib.request.urlopen('https://nanos.ai/')
article = scrapped_data .read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

article_text    

'Create optimized ad campaigns on Google, Facebook, and Instagram all in one place with Nanos patented Artificial Intelligence technologyNo website, no Facebook Business page requiredOnline marketing should be this simpleIt’s like having an agency in your pocket!Step 1: Sign up at Nanos and tell us in a few words about your great product or serviceStep 2: Nanos AI will suggest ad text, keywords, interests, and platforms for your adStep 3: Decide on the budget you want to spend, and how long to run your adsStep 4: Log in to your dashboard and watch Nanos AI in action optimizing at lightning speedWhether you have a physical store or an online venture, the goal is the same: to grow your business!We understand that existing marketing tools may be too complicated and expensive. After all, not everyone is a marketing expert or can afford to hire one.\nNo website or landing page? We’ve got you covered!At Nanos, we take the confusion out of online advertising. Nanos fees are transparent: we ch

In [ ]:

import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

def process_text(text):
  text = text.lower()
  text=text.replace('{html}',"") 
  cleanr = re.compile('<.*?>')
  text = re.sub(cleanr, '', text)
  rem_url=re.sub(r'http\S+', '',text)
  text = re.sub('[0-9]+', '', rem_url)
  
  return [token.text for token in nlp(text) if not token.is_stop]


In [ ]:
# sentences=(nltk.tokenize.sent_tokenize(article_text))
# sentences = sentences[:2]
my_list=[]
my_list.append(article_text)
mysent_dict ={}
mysent_dict['sent'] = my_list
mysent_dict

mydata = pd.DataFrame.from_dict(mysent_dict)
mydata

,sent
0,"Create optimized ad campaigns on Google, Faceb..."


In [ ]:
mydata['processed'] = mydata['sent'].map(lambda x: process_text(x))
(mydata['processed'])


0    [create, optimized, ad, campaigns, google, ,, ...
Name: processed, dtype: object

In [ ]:
from pymagnitude import Magnitude

def init_model():
  model = Magnitude('/content/drive/MyDrive/crawl-300d-2M.magnitude')
  return model

em_model = init_model()

In [ ]:
import numpy as np

def text2vec(doc_tok, model, dim=300):
    doc_embedding = np.zeros(dim)
    valid_words = 0
    # print((doc_tok))
    for word in doc_tok:
        
        if word in model:
            
            valid_words += 1
            doc_embedding += model.query(word)
        else:
            continue
    if valid_words > 0:
        return doc_embedding / valid_words
    else:
        return doc_embedding

def mytext2vec(doc_tok, model, dim=300):
    doc_embedding = np.zeros(dim)
    valid_words = 0
    # print((doc_tok))
    for word in doc_tok:
        
        if word in model:
            
            valid_words += 1
            doc_embedding += model.query(word)
        else:
            continue
    if valid_words > 0:
        return doc_embedding / valid_words
    else:
        return doc_embedding

   
        
def get_docs_embedding(docs_tok, model, dim=300):
    all_docs_embedding = []
    for doc in docs_tok:
        all_docs_embedding.append(text2vec(doc, model, dim))
    return np.array(all_docs_embedding)
    


In [ ]:
headlines_embedding = get_docs_embedding(docs_tok=data['processed_headlines'], model=em_model)


In [ ]:
my_headlines_embedding = get_docs_embedding(mydata['processed'], model=em_model)


In [ ]:
from itertools import product

def get_relevant_words(search_tok, doc_tok, model):
    search_set = set()
    doc_set = set()
    word_array = set()
    for word in search_tok:
        if word in model:
            search_set.add(word)

    for word in doc_tok:
        if word in model:
            doc_set.add(word)
            
    for s in  product(search_set, doc_set):
        if model.similarity(s[0], s[1]) >= 0.45:
            word_array.add(s[1])
    return ', '.join(list(word_array))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def semantic_search_cossim(search_string, docs_embeddings, em_model, data, topn=10):
    # clean search text
    process_search_str = process_text(search_string)
    # convert search text to vec
    search_string_vect = np.array(text2vec(process_search_str, em_model)).reshape(1, -1)
    # find cosine similarity b/w search text and document headlines
    cosine_similarities = pd.Series(cosine_similarity(search_string_vect, docs_embeddings)[0])
    # create an empty dataframe to write output to
    result_df = pd.DataFrame(columns=['relevant_words', 'similarity'], index=range(topn))
    k = 0
    # write data to output dataframe by sorting in ascending order of cosine similarity
    for i, j in cosine_similarities.nlargest(int(topn)).iteritems():
        # result_df['headlines_matched'][k] = data['sent'][i]
        result_df['relevant_words'][k] = get_relevant_words(search_tok=process_search_str,
                                                            doc_tok=data.processed[i],
                                                            model=em_model)
        result_df['similarity'][k] = j
        k += 1

    return result_df

search_headline = input("Search: ")
results = semantic_search_cossim(search_headline, my_headlines_embedding, em_model,
                                mydata, topn=1)


print("Your answere = ",list(results['relevant_words']),results['similarity'])

Search: digital marketing, digital marketing tool
Your answere =  ['useful, tools, product, sales, business, tool, ,, brand, marketing, advertising, digital'] 0    0.684314
Name: similarity, dtype: object


In [ ]:
k=(results['relevant_words'])
s=(results['similarity'])

list(k),list(s)

(['useful, tools, product, sales, business, tool, ,, brand, marketing, advertising, digital'],
 [0.6843143141181023])

In [ ]:
https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db

https://statsmaths.github.io/stat289-f18/solutions/tutorial19-gensim.html